In [15]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models,datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm 

In [22]:
# Step 1: Load the pretrained ResNet-18 model
model = models.resnet18(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 10)  # CIFAR-10 has 10 classes

In [23]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
# Step 2.0: Preprocess the ILSVRC2010 dataset
# Note: Download the dataset or point `root` to your ImageNet folder.
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_dataset = torchvision.datasets.ImageNet(root='/path/to/ILSVRC2010/', split='val', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [14]:
# Step 2.1: Preprocess the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize(224),  # ResNet expects input size of 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


Files already downloaded and verified


In [16]:
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

100%|██████████| 170498071/170498071 [00:07<00:00, 22289260.43it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [17]:
# 2.2 Fine-tuning setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
n_epochs = 3

In [18]:
# 2.3 Fine-tuning loop
model.train()
for epoch in range(n_epochs):
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}", ncols=100):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

Epoch 1:  28%|██████████████▊                                     | 222/782 [11:31<29:04,  3.11s/it]


KeyboardInterrupt: 

In [ ]:
# 4. Evaluate the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(val_loader, desc="Evaluating", ncols=100):
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test images: {accuracy:.2f}%')

In [8]:
# Step 3: Evaluate the model
correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating", ncols=100):
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test images: {accuracy:.2f}%')

Evaluating:   0%|                                                           | 0/157 [00:00<?, ?it/s]

Evaluating: 100%|█████████████████████████████████████████████████| 157/157 [02:58<00:00,  1.13s/it]

Accuracy on the test images: 0.00%


In [11]:
for images, labels in tqdm(test_loader, desc="Evaluating", ncols=100):
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    print(predicted)
    print(labels)
    break

Evaluating:   0%|                                                           | 0/157 [00:01<?, ?it/s]

tensor([841, 814, 408, 772, 364, 377, 675,  59, 285, 675, 473, 675, 167, 375,
        675, 814, 215, 339, 724,  32, 221, 596, 165, 675, 175, 158, 241,  51,
        675, 274, 335, 190, 274, 676, 675, 377, 351, 675, 675, 152, 676, 364,
        152, 364, 856, 675, 262, 427, 339,  32, 404, 510, 995, 275, 623, 914,
        371, 241, 273, 335, 339, 335,  59, 804])
tensor([3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9,
        5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9,
        7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 6, 7, 3, 6, 3])


In [19]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  